In [34]:
import re
sample_in = """
Player 1 starting position: 4
Player 2 starting position: 8
"""
my_in = """
Player 1 starting position: 6
Player 2 starting position: 1
"""


def get_input(s):
    player_matches = [
        re.search('^Player ([12]) starting position: ([0-9]+)$', x)
        for x in s.strip().split('\n')]
    players = [int(pm.groups()[1]) for pm in player_matches]
    return players


print(get_input(sample_in))
print(get_input(my_in))


[4, 8]
[6, 1]


In [35]:
def prob1(s):
    positions = get_input(s)
    finished = False
    rolls = 0
    scores = [0, 0]
    parity = 0
    while not finished:
        pos = get_points_test_dice(positions[parity], rolls)
        positions[parity] = pos
        scores[parity] += pos
        if scores[parity] >= 1000:
            finished = True
        #
        # print(rolls, positions, scores)
        parity = (parity + 1) % 2
        rolls += 3
    loser = scores[parity]
    print(scores, rolls)
    return loser * rolls


def get_points_test_dice(position, rolls):
    return ((position + ((rolls+2) * 3) - 1) % 10) + 1


test1 = prob1(sample_in)
assert test1 == 739785, test1
assert prob1(my_in) == 929625
print('ok')


[1000, 745] 993
[1006, 925] 1005
ok


In [36]:
def prob2(s):
    positions = get_input(s)
    cache = {'l': []}
    pos_key = (positions[0], 0, positions[1], 0, 0)
    cache[pos_key] = 0
    pending = {pos_key}
    i = 0
    while pending:
        i += 1
        if i % 10000 == 1:
            print(
                f'Iter {i} - Len pending: {len(pending)} - Len cache: {len(cache)}')
        add_options(cache, pending)
    vresult = [cache.get((a, b, c, d, 1), 0) for a in range(11)
               for c in range(11) for d in range(21) for b in [21, 22, 23]]
    result = sum(vresult)
    print(f'Len cache: {len(cache["l"])} result: {result}')

    return result


def add_options(cache, pending):
    pos = get_min(pending)
    x, score1, y, score2, my_turn = pos
    new_turn = 1 - my_turn
    if score1 > 21 or score2 > 20:
        return
    for a in range(1, 4):
        for b in range(1, 4):
            for c in range(1, 4):
                i = a + b + c
                if my_turn==0:
                    new_pos1 = ((x + i - 1) % 10) + 1
                    key1 = (new_pos1, score1 + new_pos1, y, score2, new_turn)
                    cache[key1] = cache[pos] + 1
                    pending.add(key1)
                else:
                    new_pos2 = ((y + i - 1) % 10) + 1
                    key2 = (x, score1, new_pos2, score2 + new_pos2, new_turn)
                    cache[key2] = cache[pos] + 1
                    pending.add(key2)


def get_min(pending):
    min_val = (21, 21, 21, 21, True)
    for p in pending:
        if gpiv(p) < gpiv(min_val):
            min_val = p
    if min_val != (21, 21, 21, 21, True):
        pending.remove(min_val)
    return min_val


def gpiv(point):
    """Get point inherent value."""
    a, b, c, d, turn = point
    return a+b+c+d+turn

if None:
    test2 = prob2(sample_in)
    assert test2 == 444356092776315, test2
    prob2(my_in)


In [37]:
def prob2(s):
    cache = {}
    position = get_input(s)
    clau = (0,0, position[0], position[1], True)
    win1, win2 = get_winners(clau, cache)
    print(position)
    print(win1, win2, max(win1, win2))
    return max(win1, win2)

def get_winners(clau, cache):
    if clau in cache:
        return cache[clau]
    score1, score2, pos1, pos2, torn1 = clau
    if score1 >= 21:
        return 1, 0
    if score2 >= 21:
        return 0, 1
    wins1 = 0
    wins2 = 0
    for a in range(1, 4):
        for b in range(1, 4):
            for c in range(1, 4):
                move = a + b + c
                if torn1:
                    new_pos = ((pos1 + move - 1) % 10) + 1
                    new_score = score1 + new_pos
                    new_clau = (new_score, score2, new_pos, pos2, not torn1)
                else:
                    new_pos = ((pos2 + move - 1) % 10) + 1
                    new_score = score2 + new_pos
                    new_clau = (score1, new_score, pos1, new_pos, not torn1)
                w1, w2 = get_winners(new_clau, cache)
                wins1 += w1
                wins2 += w2
    cache[clau] =  wins1, wins2
    return cache[clau]


test2 = prob2(sample_in)
assert test2 == 444356092776315, test2
prob2(my_in)

[4, 8]
444356092776315 341960390180808 444356092776315
[6, 1]
153087536629019 175731756652760 175731756652760


175731756652760